# <font color="green">  Optimization for Mnist <font>

## 1. Build the network

In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import _pickle as cPickle
from random import randint
from math import ceil, sqrt
from scipy.misc import imsave
from matplotlib import pyplot as plt
from tensorflow.python.platform import gfile

In [2]:
# Load MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
batchsize = 50
imagesize = 28
colors = 1

img = tf.placeholder("float",shape=[None,imagesize,imagesize,colors])
lbl = tf.placeholder("float",shape=[None,10])

In [4]:
# The reference is scratch_lec04.ipynb from https://stat.columbia.edu/~cunningham/teaching/
def conv(x, W):
    """simple wrapper for tf.nn.conv2d"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    """simple wrapper for tf.nn.max_pool with stride size 2"""
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

def norm(x): 
    """simple wrapper for tf.nn.lrn... See section 3.3 of Krizhevsky 2012 for details"""
    return tf.nn.lrn(x, depth_radius=5, bias=2, alpha=1e-4, beta=0.75)

In [5]:
# The reference is scratch_lec04.ipynb from https://stat.columbia.edu/~cunningham/teaching/

def compute_logits(x,pkeep):    
    # 2 layer cnn
    # imput image data
    img = tf.reshape(x,shape=[-1,imagesize,imagesize,colors])# batch, then width, height, channels
    
    # convolution 1
    W_conv1 = tf.get_variable('W_conv1', shape=[5,5,colors,32])
    b_conv1 = tf.get_variable('b_conv1', shape=[32])
    h_conv1 = tf.nn.relu(tf.add(conv(img, W_conv1), b_conv1))
    
    # pool 1
    h_pool1 = maxpool(h_conv1)

    # convolution 2
    W_conv2 = tf.get_variable('W_conv2', shape=[5,5,32,64])
    b_conv2 = tf.get_variable('b_conv2', shape=[64])
    h_conv2 = tf.nn.relu(tf.add(conv(h_pool1, W_conv2), b_conv2))

    # pool 2
    h_pool2 = maxpool(h_conv2)

    # fc 1
    pool2flat = tf.reshape(h_pool2, [-1, int((imagesize/4) * (imagesize/4) *64)])
    W_fc1 = tf.get_variable('W_fc1', shape=[int((imagesize/4) * (imagesize/4) * 64), 1024])
    b_fc1 = tf.get_variable('b_fc1', shape=[1024])
    h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool2flat, W_fc1), b_fc1))

    h_fc1_drop = tf.nn.dropout(h_fc1, pkeep)
    
    # fc 2
    W_fc2 = tf.get_variable('W_fc2', shape=[1024,10])
    b_fc2 = tf.get_variable('b_fc2', shape=[10])

    # logits
    logits = tf.add(tf.matmul(h_fc1_drop, W_fc2), b_fc2, name='logits')

    return logits, W_conv1, W_conv2,b_conv1,b_conv2, W_fc1, b_fc1, W_fc2, b_fc2

In [6]:
# The reference is scratch_lec04.ipynb from https://stat.columbia.edu/~cunningham/teaching/

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

## 2. Prepare the dataset

In [7]:
# Dataset preparation
rawimage =  mnist.train.images
Y_train = mnist.train.labels
X_train = np.reshape(rawimage,[-1,28,28,1])

## 3. Pretrain the network

In [8]:
# The reference is scratch_lec04.ipynb from https://stat.columbia.edu/~cunningham/teaching/

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, [None, 28*28*1], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    pkeep = tf.placeholder(tf.float32, name='pkeep')
    
    with tf.name_scope('model'):
        logits,W_conv1, W_conv2,b_conv1,b_conv2,W_fc1, b_fc1, W_fc2, b_fc2= compute_logits(x,pkeep)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):
        opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.Session() as sess:        
        sess.run(tf.global_variables_initializer())
    
        for i in range(2001):
            batch_index = np.random.randint(0,50000,batchsize)
            X_batch = X_train[batch_index,:,:,:].reshape([batchsize,-1])
            y_batch = [Y_train[i] for i in batch_index]

            # run
            _  = sess.run((train_step),feed_dict={x: X_batch, y: y_batch, pkeep:0.85})

            # print diagnostics
            if i%200 == 0:
                X_batch = X_train[0:1000,:,:,:].reshape([1000,-1])
                y_batch = Y_train[0:1000]
                (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:0.85})
                print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                
        conv1_w,conv2_w,conv1_b,conv2_b,fc1_w,fc1_b,fc2_w,fc2_b=sess.run((W_conv1,W_conv2,b_conv1,b_conv2,W_fc1, b_fc1, W_fc2, b_fc2), {x: X_train.reshape([X_train.shape[0],-1]), y: Y_train, pkeep:0.85})

Step   0: training accuracy 0.1360
Step 200: training accuracy 0.8970
Step 400: training accuracy 0.9340
Step 600: training accuracy 0.9490
Step 800: training accuracy 0.9620
Step 1000: training accuracy 0.9690
Step 1200: training accuracy 0.9690
Step 1400: training accuracy 0.9750
Step 1600: training accuracy 0.9740
Step 1800: training accuracy 0.9670
Step 2000: training accuracy 0.9780


In [9]:
# Compute the results of each layer for a given image dataset

img = tf.placeholder("float",[None,28,28,1])

# convolution 1
conv1_output = tf.nn.relu(tf.add(conv(img, conv1_w), conv1_b))

# pool 1
pool1_output = maxpool(conv1_output)

# convolution 2
conv2_output = tf.nn.relu(tf.add(conv(pool1_output, conv2_w), conv2_b))

# pool 2
pool2_output = maxpool(conv2_output)

# fc 1
pool2flat_output = tf.reshape(pool2_output, [-1, int(7 * 7 *64)])
fc1_output = tf.nn.relu(tf.add(tf.matmul(pool2flat_output, fc1_w), fc1_b))
fc1_drop_output = tf.nn.dropout(fc1_output, 0.85)

# logits
logits_output = tf.add(tf.matmul(fc1_drop_output, fc2_w), fc2_b)

# <font color="green">  Method 1.Deconvolution <font>

## 4. Build the deconvolutional network

In [10]:
# The unpool_1 function is referred from https://github.com/kvfrans/feature-visualization/blob/master/main.py
def unpool(value):
    """N-dimensional version of the unpooling operation from
    https://www.robots.ox.ac.uk/~vgg/rg/papers/Dosovitskiy_Learning_to_Generate_2015_CVPR_paper.pdf
    :param value: A Tensor of shape [b, d0, d1, ..., dn, ch]
    :return: A Tensor of shape [b, 2*d0, 2*d1, ..., 2*dn, ch]
    """
    sh = list(value.shape)
    dim = len(sh[1:-1])
    out = (tf.reshape(value, [-1] + sh[-dim:]))
    for i in range(dim, 0, -1):
        out = tf.concat([out, out],i)
    out_size = [-1] + [(s*2) for s in sh[1:-1]] + [sh[-1]]
    out = tf.reshape(out, out_size)
    with tf.Session() as sess:
        outs=out.eval()
    return outs

## 5.Take images into grid

In [12]:
# Referenced from https://github.com/InFoCusp/tf_cnnvis/blob/master/tf_cnnvis/utils.py
def convert_into_grid(Xs, ubound=255.0, padding=1):
    """
    Convert 4-D numpy array into a grid image
    :param Xs: 
        A numpy array of images to make grid out of it
    :type Xs: 4-D numpy array (first axis contations an image)
    :param ubound: 
        upperbound for a image pixel value
    :type ubound: float (Default = 255.0)
    :param padding: 
        padding size between grid cells
    :type padding: int (Default = 1)
    :return: 
        A grid of input images 
    :rtype: 3-D numpy array
    """
    (N, H, W, C) = Xs.shape
    grid_size = int(ceil(sqrt(N)))
    grid_height = H * grid_size + padding * (grid_size - 1)
    grid_width = W * grid_size + padding * (grid_size - 1)
    grid = np.zeros((grid_height, grid_width, C))
    next_idx = 0
    y0, y1 = 0, H
    for y in range(grid_size):
        x0, x1 = 0, W
        for x in range(grid_size):
            if next_idx < N:
                grid[y0:y1, x0:x1] = Xs[next_idx]
                next_idx += 1
            x0 += W + padding
            x1 += W + padding
        y0 += H + padding
        y1 += H + padding
    return grid.astype('float32')

def images_to_grid(images):
    """
    Convert a list of arrays of images into a list of grid of images
    :param images: 
        a list of 4-D numpy arrays(each containing images)
    :type images: list
    :return: 
        a list of grids which are grid representation of input images
    :rtype: list
    """
    grid_images = []
    if len(images) > 0:
        N = len(images)
        H, W, C = images[0][0].shape
        for j in range(len(images[0])):
            tmp = np.zeros((N, H, W, C))
            for i in range(N):
                tmp[i] = images[i][j]
            grid_images.append(np.expand_dims(convert_into_grid(tmp), axis = 0))
    return grid_images

def write_deconv(images,layer,path_outdir,filename):
    grid_images = images_to_grid(images)

    path_out = os.path.join(path_outdir, layer.lower().replace("/", "_"))

    for i in range(len(grid_images)):
        image_data=grid_images[i]
        resized_images=tf.image.resize_nearest_neighbor(image_data,size=[168,168])
        with tf.Session() as sess:
            resized_image_data=resized_images.eval()
        grid_image_path = os.path.join(path_out)
        os.mkdir(grid_image_path)
        if image_data.shape[-1] == 1:
            imsave(os.path.join(grid_image_path,filename+".png"),resized_image_data[0,:,:,0], format = "png")
        else:
            imsave(os.path.join(grid_image_path,filename+".png"), resized_image_data[0], format = "png")

## 6. Display the best images for each feature map in grid

In [13]:

def display(value, optimize_method):
    print("displaying")
    best_feature_maps_1=[]
    best_base_images_1=[]
    best_feature_maps_2=[]
    best_base_images_2=[]
    start_time = time.time()
    
    # Deconv 1
    # Referenced from https://github.com/kvfrans/feature-visualization/blob/master/main.py
    featuresReLu1 = tf.placeholder("float",[None,28,28,32])
    unReLu1 = tf.nn.relu(featuresReLu1)
    unBias1 = unReLu1
    unConv1 = tf.nn.conv2d_transpose(unBias1, conv1_w, output_shape=[value.shape[0],imagesize,imagesize,colors] , 
                                strides=[1,1,1,1], padding="SAME")

    # Deconv 2
    featuresReLu2 = tf.placeholder("float",[None,14,14,64])
    unReLu2 = tf.nn.relu(featuresReLu2)
    unBias2 = unReLu2
    unConv2 = tf.nn.conv2d_transpose(unBias2, conv2_w, output_shape=[value.shape[0],int(imagesize/2),int(imagesize/2),32] , 
                                strides=[1,1,1,1], padding="SAME")
     
    with tf.Session() as sess:
        pool1_result=pool1_output.eval(feed_dict={img:value})
        pool2_result=pool2_output.eval(feed_dict={img:value})
    
    # Unpool 1
    unpool1=unpool(pool1_result)

    # Unpool 2
    unpool2=unpool(pool2_result)

    # Display layer 1
    # Referenced from https://github.com/kvfrans/feature-visualization/blob/master/main.py
    for i in range(32):
        isolated = unpool1.copy()
        isolated[:,:,:,:i] = 0
        isolated[:,:,:,i+1:] = 0
        # Set all pixels of all feature maps except for i'th to 0
        # We compute the sum or variance of all pixels in each feature map for all images and choose the image with the highest one
        # This was inspired by the article https://github.com/kvfrans/feature-visualization/blob/master/main.py
        if optimize_method=="argmax_sum":
            totals = np.sum(isolated,axis=(1,2,3))
        # This was inspired by the article https://www.sciencedirect.com/science/article/pii/S1877050917323840
        elif optimize_method=="argmax_var":
            totals = np.sum(isolated,axis=(1,2,3))

        best = np.argmax(totals,axis=0)
        
        print("layer1_feature"+str(i))
        print(best)
        print("--- %s seconds ---" % (time.time() - start_time))
        with tf.Session() as sess:
            pixelactive = unConv1.eval(feed_dict={featuresReLu1: isolated})
            # Deconvolution of conv 1 of all images

        pb = pixelactive[best]
        pb = pb.reshape([1, 28, 28,1]) 
        best_feature_maps_1.append(pb)
        
        ib = value[best]
        ib = ib.reshape([1, 28, 28,1])
        best_base_images_1.append(ib)
    
    # Show best reconstructed images for all feature maps.
    write_deconv(best_base_images_1,"pool1","./saveimage/minst_layer1/best_base_images","pool1_feature_maps")
    # Show all initial images corresponding to reconstructed images above.
    write_deconv(best_feature_maps_1,"pool1","./saveimage/minst_layer1/best_feature_maps","pool1_feature_maps") 
    
        
    #Display layer 2
    for j in range(64):
        isolated2 = unpool2.copy()
        isolated2[:,:,:,:j] = 0
        isolated2[:,:,:,j+1:] = 0
        if optimize_method=="argmax_sum":
            totals = np.sum(isolated2,axis=(1,2,3))
        elif optimize_method=="argmax_var":
            totals = np.sum(isolated2,axis=(1,2,3))
        best = np.argmax(totals,axis=0)
        print(best)
        print("layer2_feature"+str(j))
        print("--- %s seconds ---" % (time.time() - start_time))
        with tf.Session() as sess:
            unConv2_active=unConv2.eval(feed_dict={featuresReLu2: isolated2})
            pixelactive = unConv1.eval(feed_dict={featuresReLu1: unpool(unConv2_active)})
        
        pb_2 = pixelactive[best]
        pb_2 = pb_2.reshape([1, 28, 28,1]) 
    
        ib_2 = value[best]
        ib_2 = ib_2.reshape([1, 28, 28,1])
        best_feature_maps_2.append(pb_2)
        best_base_images_2.append(ib_2)
        
    write_deconv(best_feature_maps_2,"pool2","./saveimage/minst_layer2/best_feature_maps","pool2_feature_maps")
    write_deconv(best_base_images_2,"pool2","./saveimage/minst_layer2/best_base_images","pool2_feature_maps")  

In [ ]:
np.random.seed(1)
#display(X_train[np.random.choice(range(X_train.shape[0]),1),:,:,:], "argmax_var")

In [ ]:
#display(X_train[np.random.choice(range(X_train.shape[0]),500,replace=False),:,:,:],  "argmax_var")

# <font color="green">  Method2. Optimization <font>

## 7. Channel/feature maps

In [14]:
#Visualize the optimized feature maps in conv1
def optimization_minst_channel_conv1(steps=500,optimizer_method="SGD"):
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_feature_conv1_1=[]
    start_time = time.time()
    
    for i in range(32):
        
        # We calculate the variance of all pixels of each feature map as the activation function.
        # This was inspired by the article https://www.sciencedirect.com/science/article/pii/S1877050917323840
        goal = tf.reduce_sum(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))[:,:,:,i:i+1],[0,1,2,3])
        goal2 = tf.reshape(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))[:,:,:,i:i+1], [-1])
        goal3 = tf.reduce_sum(tf.square(goal2 - (goal/784)))
       
        if optimizer_method=="SGD":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        if optimizer_method=="Adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        # We use 2 methods of optimizer.
        train_step = optimizer.minimize(-1 * goal3)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()
            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))

            xlast2 = xlast.reshape(1, 28, 28, 1)
            opt_feature_conv1_1.append(xlast2)
            
    write_deconv(opt_feature_conv1_1,"conv1","./optimization/minst/channel","conv1_feature_maps")
    
    return None         

In [15]:
#optimization_minst_channel_conv1(500)

In [16]:
#Visualize the optimized feature maps in pool1

def optimization_minst_channel_pool1(steps=500,optimizer_method="SGD"):
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_feature_pool_1=[]
    start_time = time.time()
    for i in range(32):
        
        goal = tf.reduce_sum(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b)))[:,:,:,i:i+1],[0,1,2,3])
        goal2 = tf.reshape(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b)))[:,:,:,i:i+1], [-1])
        goal3 = tf.reduce_sum(tf.square( goal2 - (goal/6272)))
        
        if optimizer_method=="SGD":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        if optimizer_method=="Adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_step = optimizer.minimize(-1 * goal3)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()
            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2 = xlast.reshape(1, 28, 28, 1)
            opt_feature_pool_1.append(xlast2)
            
    write_deconv(opt_feature_pool_1,"pool1","./optimization/minst/channel","pool1_feature_maps")
    return None

In [17]:
#optimization_minst_channel_pool1(500)

In [18]:
#Visualize the optimized feature maps in conv2

def optimization_minst_channel_conv2(steps=300,optimizer_method="SGD"):
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_feature_conv2=[]
    start_time = time.time()
    for i in range(64):
        
        goal = tf.reduce_sum(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b))[:,:,:,i:i+1],[0,1,2,3])
        goal2 = tf.reshape(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b))[:,:,:,i:i+1], [-1])
        goal3 = tf.reduce_sum(tf.square( goal2 - (goal/12544)))
        
        if optimizer_method=="SGD":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        if optimizer_method=="Adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_step = optimizer.minimize(-1 * goal3)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)        
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2 = xlast.reshape([1, 28, 28, 1])
            opt_feature_conv2.append(xlast2)

    write_deconv(opt_feature_conv2,"conv2","./optimization/minst/channel","conv2_feature_maps")
    
    return None

In [19]:
#optimization_minst_channel_conv2(300)

In [20]:
#Visualize the optimized feature maps in pool2

def optimization_minst_channel_pool2(steps=300,optimizer_method="SGD"):
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_feature_pool2=[]
    start_time = time.time()
    for i in range(64):
        
        goal = tf.reduce_sum( maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))[:,:,:,i:i+1],[0,1,2,3])
        goal2 = tf.reshape( maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))[:,:,:,i:i+1], [-1])
        goal3 = tf.reduce_sum(tf.square( goal2 - (goal/3136)))
        
        if optimizer_method=="SGD":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        if optimizer_method=="Adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_step = optimizer.minimize(-1 * goal3)
        with tf.Session() as sess:
            xx.initializer.run()
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i) 
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2 = xlast.reshape([1, 28, 28, 1])
            opt_feature_pool2.append(xlast2)

    write_deconv(opt_feature_pool2,"pool2","./optimization/minst/channel","pool2_feature_maps")
    return None

In [21]:
#optimization_minst_channel_pool2(300)

## 8. Layer

In [22]:
#Visualize the images that mostly activate conv1 layer

def optimization_minst_layer_conv1(steps=500,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_conv1=[]
    
    # We calculate the variance of all pixels of each layer as the activation function.
    goal = tf.reduce_sum(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))[:,:,:,:],[0,1,2,3])
    goal2 = tf.reshape(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))[:,:,:,:], [-1])
    goal3 = tf.reduce_sum(tf.square( goal2 - (goal/25088)))
    

    if optimizer_method=="SGD":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    if optimizer_method=="Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_step = optimizer.minimize(-1 * goal3)

    with tf.Session() as sess:
        for i in range(4):
            sess.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2 = xlast.reshape([1, 28, 28, 1])
            opt_layer_conv1.append(xlast2)
            
    write_deconv(opt_layer_conv1,"conv1","./optimization/minst/layer","conv1_layer_maps")

    return None

In [23]:
#optimization_minst_layer_conv1(500)

In [24]:
#Visualize the images that mostly activate pool1 layer

def optimization_minst_layer_pool1(steps=500,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_pool1=[]
    goal = tf.reduce_sum(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b)))[:,:,:,:],[0,1,2,3])
    goal2 = tf.reshape(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))[:,:,:,:], [-1])
    goal3 = tf.reduce_sum(tf.square( goal2 - (goal/6272)))
    
    if optimizer_method=="SGD":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    if optimizer_method=="Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_step = optimizer.minimize(-1 * goal3)
    
    with tf.Session() as sess:
        for i in range(4):
            ss.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2=xlast.reshape(28,28)
            opt_layer_pool1.append(xlast2)

    write_deconv(opt_layer_pool1,"pool1","./optimization/minst/layer","pool1_layer_maps")

    return None

In [25]:
#optimization_minst_layer_pool1(500)

In [26]:
#Visualize the images that mostly activate conv2 layer

def optimization_minst_layer_conv2(steps=300,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_conv2=[]
    
    goal = tf.reduce_sum(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b))[:,:,:,:],[0,1,2,3])
    goal2 = tf.reshape(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b))[:,:,:,:], [-1])
    goal3 = tf.reduce_sum(tf.square( goal2 - (goal/12544)))
    
    if optimizer_method=="SGD":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    if optimizer_method=="Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_step = optimizer.minimize(-1 * goal3)
    
    with tf.Session() as sess:
        for i in range(4):
            ss.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2=xlast.reshape(28,28)
            opt_layer_conv2.append(xlast2)
    
    write_deconv(opt_layer_conv2,"conv2","./optimization/minst/layer","conv2_layer_maps")

    return None

In [27]:
#optimization_minst_layer_conv2(300)

In [28]:
#Visualize the images that mostly activate pool2 layer

def optimization_minst_layer_pool2(steps=300,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_pool2=[]
    goal = tf.reduce_sum( maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))[:,:,:,:],[0,1,2,3])
    goal2 = tf.reshape( maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))[:,:,:,:], [-1])
    goal3 = tf.reduce_sum(tf.square( goal2 - (goal/3136)))
    if optimizer_method=="SGD":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    if optimizer_method=="Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_step = optimizer.minimize(-1 * goal3)
    
    with tf.Session() as sess:
        for i in range(4):
            ss.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2=xlast.reshape(28,28)
            opt_layer_pool2.append(xlast2)
    
    write_deconv(opt_layer_pool2,"pool2","./optimization/minst/layer","pool2_layer_maps")

    return None

In [29]:
#optimization_minst_layer_pool2(300)

In [30]:
#Visualize the images that mostly activate fc1 layer

def optimization_minst_layer_fc1(steps=300,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_fc1=[]
    
    pool2_output = maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))
    pool2flat_output = tf.reshape(pool2_output, [-1, int(7 * 7 *64)])
    goal = tf.reduce_sum(tf.nn.relu(tf.add(tf.matmul(pool2flat_output, fc1_w), fc1_b))[:,:],[0,1])
    goal2 = tf.reshape(tf.nn.relu(tf.add(tf.matmul(pool2flat_output, fc1_w), fc1_b))[:,:], [-1])
    goal3 = tf.reduce_sum(tf.square( goal2 - (goal/1024)))
    
    if optimizer_method=="SGD":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    if optimizer_method=="Adam":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_step = optimizer.minimize(-1 * goal3)
    
    with tf.Session() as sess:
        for i in range(4):
            ss.run(tf.global_variables_initializer())
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2=xlast.reshape(28,28)
            opt_layer_fc1.append(xlast2)
    
    write_deconv(opt_layer_fc1,"fc1","./optimization/minst/layer","fc1_layer_maps")

    return None

In [31]:
#optimization_minst_layer_fc1(300)

## 9. Logits

In [32]:
#Visualize the image that mostly activate the logits

def optimization_minst_logits(steps=500,optimizer_method="SGD"):
    start_time = time.time()
    xx = tf.Variable(tf.random_uniform([1,28,28,1], minval=0.0, maxval=1.0, dtype=tf.float32, name="xx"))  
    opt_layer_logits=[]
    for i in range(10):
        
        pool2_output = maxpool(tf.nn.relu(tf.add(conv(maxpool(tf.nn.relu(tf.add(conv(xx, conv1_w), conv1_b))), conv2_w), conv2_b)))
        pool2flat_output = tf.reshape(pool2_output, [-1, int(7 * 7 *64)])
        fc1_output = tf.nn.relu(tf.add(tf.matmul(pool2flat_output, fc1_w), fc1_b))#1,1024
        fc1_drop_output = tf.nn.dropout(fc1_output, 0.85)
        goal = tf.add(tf.matmul(fc1_drop_output, fc2_w), fc2_b)[0,i:i+1]
        # We take the logits of each calss as the activation function.

        if optimizer_method=="SGD":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        if optimizer_method=="Adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
        train_step = optimizer.minimize(-1 * goal3)
        with tf.Session() as sess:
            xx.initializer.run()
            for j in range(steps):
                train_step.run()

            xlast = xx.eval()
            print(i)
            print("--- %s seconds ---" % (time.time() - start_time))
            xlast2=xlast.reshape(28,28)
            opt_layer_logits.append(xlast2)
    
    write_deconv(opt_layer_logits,"logits","./optimization/minst/logits","logits_layer_maps")

    return None

In [33]:
#optimization_minst_logits(500)